# Prueba funcionamiento BeautifulSoup
---
- _Parsear_: analizar y convertir una entrada en un formato interno que el entorno de ejecución pueda realmente manejar.

El paquete Beautiful Soup es ampliamente utilizado en técnicas de «scraping» permitiendo «parsear» principalmente código HTML.

In [1]:
from bs4 import BeautifulSoup
import re

## Haciendo la sopa
---
Para empezar a trabajar con Beautiful Soup es necesario construir un objeto de tipo BeautifulSoup que reciba el contenido a «parsear»:

In [2]:
contents = """

<html lang="en">

<head>

    <title>Just testing</title>

</head>

<body>

    <h1>Just testing</h1>

    <div class="block">

      <h2>Some links</h2>

      <p>Hi there!</p>

      <ul id="data">

        <li class="blue"><a href="https://example1.com">Example 1</a></li>

        <li class="red"><a href="https://example2.com">Example 2</a></li>

        <li class="gold"><a href="https://example3.com">Example 3</a></li>

      </ul>

    </div>

    <div class="block">

      <h2>Formulario</h2>

      <form action="" method="post">

        <label for="POST-name">Nombre:</label>

        <input id="POST-name" type="text" name="name">

        <input type="submit" value="Save">

      </form>

    </div>

    <div class="footer">

      This is the footer

      <span class="inline"><p>This is span 1</p></span>

      <span class="inline"><p>This is span 2</p></span>

      <span class="inline"><p>This is span 2</p></span>

    </div>

</body>

</html>

"""

In [3]:
soup = BeautifulSoup(contents, features='html.parser')

## Localizar elementos (find_all)
---
- Localizar todos los enlaces (`a`):

In [4]:
soup.find_all('a')

[<a href="https://example1.com">Example 1</a>,
 <a href="https://example2.com">Example 2</a>,
 <a href="https://example3.com">Example 3</a>]

- Localizar todos los elementos con la clase `inline`:

In [5]:
soup.find_all(class_='inline')

[<span class="inline"><p>This is span 1</p></span>,
 <span class="inline"><p>This is span 2</p></span>,
 <span class="inline"><p>This is span 2</p></span>]

- Localizar todos los «divs» con la clase `footer`:

In [6]:
soup.find_all('div', class_='footer')  # ≈ soup.find_all('div', 'footer')

[<div class="footer">
 
       This is the footer
 
       <span class="inline"><p>This is span 1</p></span>
 <span class="inline"><p>This is span 2</p></span>
 <span class="inline"><p>This is span 2</p></span>
 </div>]

- Localizar todos los elementos cuyo atributo `type` tenga el valor `text`:

In [7]:
soup.find_all(type='text')

[<input id="POST-name" name="name" type="text"/>]

- Localizar todos los los `h2` que contengan el texto `Formulario`:

In [21]:
soup.find_all('h2', string='Formulario')

[<h2>Formulario</h2>]

- Localizar todos los elementos de título `h1, h2, h3, ...`. Esto lo podemos atacar usando _expresiones regulares_:

In [9]:
soup.find_all(re.compile(r'^h\d+.*'))

[<h1>Just testing</h1>, <h2>Some links</h2>, <h2>Formulario</h2>]

- Localizar todos los «input» y todos los «span»:

In [10]:
soup.find_all(['input', 'span'])

[<input id="POST-name" name="name" type="text"/>,
 <input type="submit" value="Save"/>,
 <span class="inline"><p>This is span 1</p></span>,
 <span class="inline"><p>This is span 2</p></span>,
 <span class="inline"><p>This is span 2</p></span>]

- Localizar todos los párrafos que están dentro del pie de página (*usando selectores CSS*):

In [11]:
soup.select('.footer p') #obtiene los parrafos (p) dl div con class footer

[<p>This is span 1</p>, <p>This is span 2</p>, <p>This is span 2</p>]

## Localizar único elemento (find)
---
Beautiful Soup nos proporciona la función `find()` que trata de *localizar un único elemento*. Hay que tener en cuenta dos circunstancias:
- En caso de que el elemento buscado no exista, se devuelve *None*.
- En caso de que existan múltiples elementos, se devuelve el primero.


In [12]:
soup.find('form')

<form action="" method="post">
<label for="POST-name">Nombre:</label>
<input id="POST-name" name="name" type="text"/>
<input type="submit" value="Save"/>
</form>

In [13]:
# Elemento que no existe
soup.find('strange-tag')

In [23]:
# Múltiples "li". Sólo se devuelve el primero
soup.find('li')

<li class="blue"><a href="https://example1.com">Example 1</a></li>

## Localizar desde elemento
---
Todas las búsquedas se pueden realizar desde cualquier elemento preexistente, no únicamente desde la raíz del DOM.
Veamos un ejemplo de ello. 

Si tratamos de *localizar todos los títulos «h2»* vamos a encontrar dos de ellos:

In [15]:
soup.find_all('h2')

[<h2>Some links</h2>, <h2>Formulario</h2>]

Pero si, previamente, nos ubicamos en el segundo bloque de contenido, sólo vamos a encontrar uno de ellos:

In [16]:
second_block = soup.find_all('div', 'block')[1] # con el numero de los corchetes indicas el numero 
#del bloque al que quieres ir 0 para el primero, 1 para el segundo, etc.
second_block

<div class="block">
<h2>Formulario</h2>
<form action="" method="post">
<label for="POST-name">Nombre:</label>
<input id="POST-name" name="name" type="text"/>
<input type="submit" value="Save"/>
</form>
</div>

In [17]:
second_block.find_all('h2')

[<h2>Formulario</h2>]

## Otras funciones de búsqueda
---
Hay definidas una serie de funciones adicionales de búsqueda para cuestiones más particulares:
- Localizar los *«div» superiores* a partir de un elemento concreto:

In [18]:
gold = soup.find('li', 'gold')
gold

<li class="gold"><a href="https://example3.com">Example 3</a></li>

In [19]:
gold.find_parents('div')

[<div class="block">
 <h2>Some links</h2>
 <p>Hi there!</p>
 <ul id="data">
 <li class="blue"><a href="https://example1.com">Example 1</a></li>
 <li class="red"><a href="https://example2.com">Example 2</a></li>
 <li class="gold"><a href="https://example3.com">Example 3</a></li>
 </ul>
 </div>]

Se podría decir que la función `find_all()` busca en _descendientes_ y que la función `find_parents()` busca en _ascendientes_.

(También existe la versión de esta función que devuelve un _único elemento_: `find_parent()`)

- Localizar los *elementos hermanos siguientes* a uno dado:

In [24]:
blue_li = soup.find('li', 'blue')
blue_li

<li class="blue"><a href="https://example1.com">Example 1</a></li>

In [25]:
blue_li.find_next_siblings()

[<li class="red"><a href="https://example2.com">Example 2</a></li>,
 <li class="gold"><a href="https://example3.com">Example 3</a></li>]

(También existe la versión de esta función que devuelve un único elemento: `find_next_sibling()`)
- Localizar los _elementos hermanos anteriores_ a uno dado:

In [26]:
gold_li = soup.find('li', 'gold')
gold_li

<li class="gold"><a href="https://example3.com">Example 3</a></li>

In [27]:

gold_li.find_previous_siblings()

[<li class="red"><a href="https://example2.com">Example 2</a></li>,
 <li class="blue"><a href="https://example1.com">Example 1</a></li>]

(También existe la versión de esta función que devuelve un único elemento: `find_previous_sibling()`)
- Localizar _todos los elementos a continuación_ de uno dado:

In [28]:
submit = soup.find('input', type='submit')
submit

<input type="submit" value="Save"/>

In [29]:

submit.find_all_next()

[<div class="footer">
 
       This is the footer
 
       <span class="inline"><p>This is span 1</p></span>
 <span class="inline"><p>This is span 2</p></span>
 <span class="inline"><p>This is span 2</p></span>
 </div>,
 <span class="inline"><p>This is span 1</p></span>,
 <p>This is span 1</p>,
 <span class="inline"><p>This is span 2</p></span>,
 <p>This is span 2</p>,
 <span class="inline"><p>This is span 2</p></span>,
 <p>This is span 2</p>]

(También existe la versión de esta función que devuelve un único elemento: `find_next()`)
- Localizar todos los elementos previos a uno dado:

In [31]:
ul_data = soup.find('ul', id='data')
ul_data

<ul id="data">
<li class="blue"><a href="https://example1.com">Example 1</a></li>
<li class="red"><a href="https://example2.com">Example 2</a></li>
<li class="gold"><a href="https://example3.com">Example 3</a></li>
</ul>

In [37]:
ul_data.find_previous() #Devuelve un único elemento anterior

<p>Hi there!</p>

In [38]:
ul_data.find_all_previous(['h1', 'h2']) #Devuelve todos los elementos anteriores que coinciden con los
# indicados por parametros

[<h2>Some links</h2>, <h1>Just testing</h1>]

Si hubiéramos hecho esta búsqueda usando `find_parents()` no habríamos obtenido el mismo resultado ya que los elementos de título no son elementos superiores de «ul»:

In [39]:
ul_data.find_parents(['h1', 'h2'])

[]

## Acceder al contenido
---
Simplificando, podríamos decir que cada elemento de la famosa «sopa» de _Beautiful Soup_ puede ser un `bs4.element.Tag` o un «string».
Para el caso de los «tags» existe la posibilidad de acceder a su contenido, al nombre del elemento o a sus atributos.
- Nombre de etiqueta: Podemos conocer el nombre de la etiqueta de un elemento usando el atributo `name`

In [40]:
soup.name

'[document]'

In [43]:
elem = soup.find('ul', id='data')
elem.name

'ul'

In [45]:
elem = soup.find('h1')
elem.name

'h1'

(Es posible modificar el nombre de una etiqueta con una simple asignación.)
- Acceso a atributos: los atributos de un elemento están disponibles como claves de un diccionario.

In [46]:
elem = soup.find('input', id='POST-name')
elem

<input id="POST-name" name="name" type="text"/>

In [47]:
elem['id']

'POST-name'

In [48]:
elem['name']

'name'

In [49]:
elem['type']

'text'

Exite una forma de acceder al diccionario completo de atributos:

In [50]:
elem.attrs

{'id': 'POST-name', 'type': 'text', 'name': 'name'}

(Es posible modificar el valor de un atributo con una simple asignación.)